In [6]:
random_state = 20
import pandas as pd
import json
def get_index_train_test(pipeline):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
    df = df.dropna()
    textos = df['text']
    labels = df['label']
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_enc = le.transform(labels)
    textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
    return (list(textos_train.index),list(textos_test.index))


def get_df_test_pred(pipeline):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
    df = df.dropna()
    textos = df['text']
    labels = df['label']
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_enc = le.transform(labels)
    textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
    df_test_pred = pd.read_csv(f'Arquivos TCC/test_set_predictions/{pipeline}.csv', sep=',')
    if dataset ==  'TC02_20news':
        df_test_pred['sentence'] = textos_test.to_list()
        df_test_pred['test_label'] = le.inverse_transform(df_test_pred['test_label'])
        df_test_pred['pred_label'] = le.inverse_transform(df_test_pred['pred_label'])
    if dataset == 'TC14_wos':
        with open("wos_label_map.json", "r") as read_file:
            label_map = json.load(read_file)
        df_test_pred['sentence'] = textos_test.to_list()  
        df_test_pred['test_label'] = df_test_pred['test_label'].apply(lambda x : label_map[str(x)])
        df_test_pred['pred_label'] = df_test_pred['pred_label'].apply(lambda x : label_map[str(x)])
    df_test_pred['index_dataset'] = get_index_train_test(pipeline)[1]

    #print(labels)
    return df_test_pred

In [7]:
best_pipes = ["TC02_20news+TFIDF+SVC","TC14_wos+TFIDF+RF"]
df_test_pred = get_df_test_pred(best_pipes[1])
df_test_pred.head(10)

,test_label,pred_label,sentence,index_dataset
0,Social cognition,Social cognition,It was recently proposed that the neural subst...,1660
1,Genetics,Immunology,Kawasaki disease is a multisystemic vasculitis...,7526
2,Genetics,Genetics,The aim of the present study was to understand...,9144
3,Digital control,Digital control,This paper explores the large-signal and small...,10065
4,Digital control,Digital control,Design and implementation of a low cost grid-c...,10613
5,Person perception,Person perception,Women are less likely than men to engage in se...,3542
6,Molecular biology,Molecular biology,This study aims to investigate the impacts of ...,9068
7,Social cognition,Nonverbal communication,The purpose of this study was to examine the u...,2716
8,Person perception,Prejudice,Sexual harassment and coercion have mainly bee...,5627
9,Digital control,Digital control,This paper presents a reference modification m...,3194


In [8]:
df_error = df_test_pred[df_test_pred['test_label']!=df_test_pred['pred_label']]
top_3_error_labels = df_error['test_label'].value_counts()[:3].index.tolist()
for label in top_3_error_labels:
    n = 5
    df_error_label = df_error[df_error['test_label']==label]
    top_n_misses = df_error_label['pred_label'].value_counts()[:n]
    print(f'True label: {label}')
    print('Most commom misunderstandings')
    print(top_n_misses)
    print('==========================')

True label: Molecular biology
Most commom misunderstandings
Cell biology        8
Immunology          1
Human Metabolism    1
Genetics            1
Name: pred_label, dtype: int64
True label: Social cognition
Most commom misunderstandings
Nonverbal communication    3
Prosocial behavior         3
Prejudice                  3
Anxiety                    1
Alzheimer's Disease        1
Name: pred_label, dtype: int64
True label: Genetics
Most commom misunderstandings
Molecular biology         4
Human Metabolism          3
Immunology                1
Alzheimer's Disease       1
Ankylosing Spondylitis    1
Name: pred_label, dtype: int64


In [9]:
pd.options.display.max_colwidth = 300
df_dropado = df_test_pred.drop_duplicates(subset=['test_label'], keep='first')
df_dropado.sort_values(by=['test_label']).tail(10)


,test_label,pred_label,sentence,index_dataset
20,Operating systems,Operating systems,"The reproducibility of experiments is one of the main principles of the scientific method. However, numerical N-body experiments, especially those of planetary systems, are currently not reproducible. In themost optimistic scenario, they can only be replicated in an approximate or statistical se...",9035
5,Person perception,Person perception,"Women are less likely than men to engage in sexually agentic behavior (e.g., initiating sexual encounters), despite the benefits associated with sexual agency (Kiefer & Sanchez, 2007). Two studies examined possible explanations, related to person perception, for gender differences in sexually ag...",3542
29,Prejudice,Prejudice,"Prior research indicates that there may be a disharmonious relationship between positive attitudes toward ethnic and cultural diversity and social identity within a socially dominant group. Recent work in cultural psychology, however, has implied that this disharmonious relationship may be confi...",10991
17,Prosocial behavior,Prosocial behavior,"This study examined the role of children's trust beliefs and trustworthiness in the development of prosocial behavior using data from four waves of a longitudinal study in a large, ethnically-diverse sample of children in Switzerland (mean age = 8.11 years at Time 1, N = 1,028). Prosocial behavi...",9168
15,Rainwater Harvesting,Rainwater Harvesting,"In comparison to other sources of water supply, rainwater harvesting (RWH) has the typical advantages of being cheaper and easier to operate and maintain. This study aimed at assessing the hydrologic rainwater harvesting potential of rural schools in South Africa by obtaining RWH storage capacit...",5367
13,Remote Sensing,Remote Sensing,Soil moisture distributions with high spatio-temporal resolution are scarce but beneficial for understanding ecohydrological processes and closing the water cycle at the basin scale. Sensor networks are innovative in their ability to capture the spatio-temporal heterogeneity and dynamics of soil...,5427
0,Social cognition,Social cognition,"It was recently proposed that the neural substrate mediating smile production might play a key role also in the recognition of others' smile. This hypothesis, however, has been challenged by difficulties in eliciting ecological smiling in standard laboratory settings. Here we report of a case wh...",1660
31,Water Pollution,Water Pollution,"The bivalve mollusks are among the aquatic bioindicators that are commonly used in monitoring water pollution studies, thanks to their behavior and metabolism. They are directly affected by the level of pollution in a given site. During this research, the study of the biological response in gill...",7163
90,computer-aided design,Electrical circuits,"In this paper, we present morphological and electrical characteristics of a junction formed of Si p-type films deposited on an n-type silicon wafer using a hot wire chemical vapor deposition (HWCVD) tool. We describe the fabrication process and study the influence of diborane flow and postproces...",158
24,network security,network security,"Facing with plethora network security events, network security risk evaluation is an effective method to prevent and respond to these security problems. Ignoring possible changes maybe occurred, traditional network security risk evaluation only emphasizes the preventive management in advance. Th...",10049


In [10]:
data_catalog = pd.read_excel('/home/vinicius/Pré Processamento/Meta-data/Data.xlsx')

In [11]:
data_catalog.columns

Index(['Y1', 'Y2', 'Y', 'Domain', 'area', 'keywords', 'Abstract'], dtype='object')

In [75]:
label_map = {}
for row_index in range(len(df_dropado)):
    label_new = int(df_dropado['test_label'].values[row_index])
    try:
        label_wos = data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][1:50].strip())]['area'].values[0]
        label_map[label_new] = label_wos.strip()
    except:
        print(f'Não achou o label_wos do {label_new}')
        print("Adicionado manualmente")

label_map[3] = 'Cryptography'
label_map[23] = 'Addiction'

/tmp/ipykernel_260/3465581413.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  label_wos = data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][1:50].strip())]['area'].values[0]


Não achou o label_wos do 3
Adicionado manualmente
Não achou o label_wos do 23
Adicionado manualmente


In [76]:
label_map
with open('wos_label_map.json', 'w') as fp:
    json.dump(label_map, fp)

In [27]:
df_dropado['test_label'].values[row_index]

7

In [32]:
from sklearn import preprocessing
df = pd.read_csv(f'Arquivos TCC/datasets/TC02_20news.csv', sep=';')
df = df.dropna()
textos = df['text']
labels = df['label']
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_enc = le.transform(labels)
label_map_20news = {}
for i,label in enumerate(labels.unique()):
    label_map_20news[i] = label

with open('20news_label_map.json', 'w') as fp:
    json.dump(label_map_20news, fp)

2

In [41]:
df_dropado['sentence'].iloc[row_index][0:100]

'This paper explores the large-signal and small-signal dynamics of a series-capacitor (SC) buck-type '

In [43]:
data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][4:50])]['area']

40638     Digital control  
Name: area, dtype: object